In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib

# Устанавливаем бэкенд для Matplotlib, чтобы избежать проблем с GUI
# (Если вы в Jupyter/Colab, эта строка, скорее всего, не нужна)
matplotlib.use('Agg')

print("--- Лабораторная работа №2 ---")

# --- ПОДГОТОВКА ДАННЫХ ---
try:
    N = 3
    print(f"Номер в списке N = {N}")

    df = pd.read_csv("students_simple.csv")

    # Выбор столбцов
    idx1 = N % 5
    idx2 = (N**2) % 5 + 5

    col_name_1 = df.columns[idx1]
    col_name_2 = df.columns[idx2]

    print(f"Выбраны столбцы:")
    print(f"  Столбец 1 (индекс {idx1}): {col_name_1}")
    print(f"  Столбец 2 (индекс {idx2}): {col_name_2}")

    # Очистка данных
    df_clean = df[[col_name_1, col_name_2]].dropna()
    x = df_clean[col_name_1].astype(float)
    y = df_clean[col_name_2].astype(float)
    n = len(df_clean)
    print(f"Размер выборки после удаления пропусков (NaN): {n}")

    # --- 1. РАСЧЕТ КОРРЕЛЯЦИЙ ---
    print("\n--- 1. Расчет корреляций ---")

    # 1.1. Коэффициент Фехнера
    med_x = np.median(x)
    med_y = np.median(y)
    signs_x = np.sign(x - med_x)
    signs_y = np.sign(y - med_y)
    # Исключаем пары, где значение равно медиане (знак = 0)
    valid_signs = (signs_x != 0) & (signs_y != 0)
    concordant = np.sum(signs_x[valid_signs] * signs_y[valid_signs] > 0)
    discordant = np.sum(signs_x[valid_signs] * signs_y[valid_signs] < 0)

    if (concordant + discordant) > 0:
        fechner_corr = (concordant - discordant) / (concordant + discordant)
        print(f"1. Коэффициент Фехнера: {fechner_corr:.4f}")
    else:
        print("1. Коэффициент Фехнера: не удалось рассчитать (нет данных).")

    # 1.2. Коэффициент Пирсона + ДИ
    pearson_r, pearson_p = stats.pearsonr(x, y)
    print(f"2. Коэффициент Пирсона: r = {pearson_r:.4f}, p-value = {pearson_p:.4f}")

    # Расчет 95% доверительного интервала
    if n > 3:
        r_z = np.arctanh(pearson_r)
        se = 1 / np.sqrt(n - 3)
        z_val = 1.96 # 95% CI
        ci_low_z, ci_high_z = r_z - z_val * se, r_z + z_val * se
        ci_low_r, ci_high_r = np.tanh(ci_low_z), np.tanh(ci_high_z)
        print(f"   95% доверительный интервал для Пирсона: [{ci_low_r:.4f}, {ci_high_r:.4f}]")
    else:
        print("   Недостаточно данных для расчета доверительного интервала (n <= 3).")

    # 1.3. Коэффициент Спирмена
    spearman_r, spearman_p = stats.spearmanr(x, y)
    print(f"3. Коэффициент Спирмена: r = {spearman_r:.4f}, p-value = {spearman_p:.4f}")

    # 1.4. Коэффициент Кенделла
    kendall_tau, kendall_p = stats.kendalltau(x, y)
    print(f"4. Коэффициент Кенделла: tau = {kendall_tau:.4f}, p-value = {kendall_p:.4f}")


    # --- 2. ВИЗУАЛИЗАЦИЯ ---
    print("\n--- 2. Визуализация ---")

    # 2.1. Гистограммы
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.hist(x, bins=20, color='blue', alpha=0.7, edgecolor='black')
    plt.title(f"Гистограмма для {col_name_1}")
    plt.xlabel(col_name_1)
    plt.ylabel("Частота")
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.subplot(1, 2, 2)
    plt.hist(y, bins=20, color='green', alpha=0.7, edgecolor='black')
    plt.title(f"Гистограмма для {col_name_2}")
    plt.xlabel(col_name_2)
    plt.ylabel("Частота")
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.tight_layout()
    plt.savefig("histograms.png")
    print("Гистограммы сохранены в 'histograms.png'.")
    # plt.show() # Раскомментируйте, если запускаете локально

    # 2.2. График рассеяния
    plt.figure(figsize=(8, 6))
    plt.scatter(x, y, alpha=0.6)
    plt.title(f"График рассеяния: {col_name_1} vs {col_name_2}")
    plt.xlabel(col_name_1)
    plt.ylabel(col_name_2)
    plt.grid(True, linestyle='--', alpha=0.6)

    plt.savefig("scatter_plot.png")
    print("График рассеяния сохранен в 'scatter_plot.png'.")
    # plt.show() # Раскомментируйте, если запускаете локально


    # --- 3. УРАВНЕНИЕ РЕГРЕССИИ ---
    print("\n--- 3. Уравнение регрессии ---")

    results = {}
    SST = np.sum((y - np.mean(y))**2) # Total Sum of Squares

    # 3.1. Линейная (ручной расчет)
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    w1_lin = np.sum((x - x_mean) * (y - y_mean)) / np.sum((x - x_mean)**2)
    w0_lin = y_mean - w1_lin * x_mean
    y_pred_lin = w1_lin * x + w0_lin
    sse_lin = np.sum((y - y_pred_lin)**2)
    results['linear'] = {'w1': w1_lin, 'w0': w0_lin, 'sse': sse_lin, 'k': 1, 'preds': y_pred_lin}
    print(f"1. Линейная: y = {w1_lin:.4f} * x + {w0_lin:.4f}")
    print(f"   SSE (Сумма квадратов ошибок): {sse_lin:.2f}")

    # 3.2. Квадратичная
    w2_quad, w1_quad, w0_quad = np.polyfit(x, y, 2)
    y_pred_quad = w2_quad * x**2 + w1_quad * x + w0_quad
    sse_quad = np.sum((y - y_pred_quad)**2)
    results['quadratic'] = {'w2': w2_quad, 'w1': w1_quad, 'w0': w0_quad, 'sse': sse_quad, 'k': 2, 'preds': y_pred_quad}
    print(f"2. Квадратичная: y = {w2_quad:.4f} * x^2 + {w1_quad:.4f} * x + {w0_quad:.4f}")
    print(f"   SSE: {sse_quad:.2f}")

    # 3.3. Гиперболическая
    # Проверка на наличие нуля в x, чтобы избежать деления
    if np.any(x == 0):
        print("3. Гиперболическая: пропущена, т.к. x содержит 0.")
    else:
        x_inv = 1 / x
        w1_hyp, w0_hyp = np.polyfit(x_inv, y, 1)
        y_pred_hyp = w1_hyp * (1/x) + w0_hyp
        sse_hyp = np.sum((y - y_pred_hyp)**2)
        results['hyperbolic'] = {'w1': w1_hyp, 'w0': w0_hyp, 'sse': sse_hyp, 'k': 1, 'preds': y_pred_hyp}
        print(f"3. Гиперболическая: y = {w1_hyp:.4f} / x + {w0_hyp:.4f}")
        print(f"   SSE: {sse_hyp:.2f}")

    # 3.4. Показательная (экспоненциальная)
    # Используем линеаризацию: log(y) = log(w0) + x*log(w1)
    # Это работает, только если все y > 0
    if np.all(y > 0):
        log_y = np.log(y)
        # m = log(w1), c = log(w0)
        m, c = np.polyfit(x, log_y, 1)
        w1_exp = np.exp(m)
        w0_exp = np.exp(c)
        y_pred_exp = w0_exp * (w1_exp ** x)
        sse_exp = np.sum((y - y_pred_exp)**2)
        results['exponential'] = {'w1': w1_exp, 'w0': w0_exp, 'sse': sse_exp, 'k': 1, 'preds': y_pred_exp}
        print(f"4. Показательная: y = {w0_exp:.4f} * ({w1_exp:.4f} ^ x)")
        print(f"   SSE: {sse_exp:.2f}")
    else:
        print("4. Показательная: пропущена, т.к. y содержит не-положительные значения.")

    # Построение графика регрессий
    plt.figure(figsize=(10, 7))
    plt.scatter(x, y, alpha=0.5, label="Исходные данные")

    # Сортируем x для плавных линий регрессии
    x_line = np.linspace(np.min(x), np.max(x), 200)

    # Линейная
    plt.plot(x_line, w1_lin * x_line + w0_lin, color='red', label=f"Линейная (SSE={sse_lin:.2f})")

    # Квадратичная
    plt.plot(x_line, w2_quad * x_line**2 + w1_quad * x_line + w0_quad, color='green', label=f"Квадратичная (SSE={sse_quad:.2f})")

    # Гиперболическая
    if 'hyperbolic' in results:
        res = results['hyperbolic']
        # Убедимся, что не делим на ноль, если x_line его пересекает
        x_line_hyp = x_line[x_line != 0]
        y_line_hyp = res['w1'] / x_line_hyp + res['w0']
        plt.plot(x_line_hyp, y_line_hyp, color='purple', label=f"Гиперболическая (SSE={res['sse']:.2f})")

    # Показательная
    if 'exponential' in results:
        res = results['exponential']
        plt.plot(x_line, res['w0'] * (res['w1'] ** x_line), color='orange', label=f"Показательная (SSE={res['sse']:.2f})")

    plt.legend()
    plt.title("Модели регрессии")
    plt.xlabel(col_name_1)
    plt.ylabel(col_name_2)
    plt.grid(True, linestyle='--', alpha=0.6)

    # Ограничим Y для лучшей читаемости, если есть выбросы
    if np.min(y) > 0:
         plt.ylim(np.min(y) * 0.9, np.max(y) * 1.1)

    plt.savefig("regression_plot.png")
    print("График регрессий сохранен в 'regression_plot.png'.")
    # plt.show() # Раскомментируйте, если запускаете локально


    # --- 4. ПРОВЕРКА УРАВНЕНИЯ РЕГРЕССИИ ---
    print("\n--- 4. Проверка уравнения регрессии ---")

    if not results:
        print("Нет моделей для оценки.")
    else:
        best_model_name = min(results, key=lambda k: results[k]['sse'])
        worst_model_name = max(results, key=lambda k: results[k]['sse'])

        print(f"Лучшая модель (мин. SSE): {best_model_name} (SSE = {results[best_model_name]['sse']:.2f})")
        print(f"Худшая модель (макс. SSE): {worst_model_name} (SSE = {results[worst_model_name]['sse']:.2f})")

        # Функция для F-теста
        def run_f_test(model_name, model_data, n_obs, S_total):
            print(f"\n  Критерий Фишера для '{model_name}' модели:")
            k = model_data['k'] # число предикторов
            sse = model_data['sse'] # Sum of Squares Error

            # R^2 = 1 - SSE / SST
            R_squared = 1.0 - (sse / S_total)

            df_model = k
            df_error = n_obs - k - 1

            if df_error <= 0:
                print(f"    Невозможно рассчитать F-тест: (n - k - 1) = {df_error} <= 0.")
                return

            # F = (R^2 / k) / ((1 - R^2) / (n - k - 1))
            # Избегаем деления на ноль, если R^2 = 1.0
            if (1.0 - R_squared) == 0:
                F_stat = float('inf')
                p_value = 0.0
            else:
                F_stat = (R_squared / df_model) / ((1.0 - R_squared) / df_error)
                p_value = stats.f.sf(F_stat, df_model, df_error) # p-value

            print(f"    R^2 = {R_squared:.4f}")
            print(f"    Степени свободы: k = {df_model} (модель), n-k-1 = {df_error} (ошибка)")
            print(f"    F-статистика = {F_stat:.4f}")
            print(f"    p-value = {p_value:.4g}")

            if p_value < 0.05:
                print("    Вывод: Модель статистически значима (p < 0.05).")
            else:
                print("    Вывод: Модель статистически не значима (p >= 0.05).")

        # F-тест для лучшей модели
        run_f_test(best_model_name, results[best_model_name], n, SST)

        # F-тест для худшей модели
        run_f_test(worst_model_name, results[worst_model_name], n, SST)

except Exception as e:
    print(f"\n--- ПРОИЗОШЛА ОШИБКА ---")
    print(e)
    import traceback
    traceback.print_exc()

print("\n--- Выполнение завершено ---")

--- Лабораторная работа №2 ---
Номер в списке N = 3
Выбраны столбцы:
  Столбец 1 (индекс 3): iq
  Столбец 2 (индекс 9): test_time
Размер выборки после удаления пропусков (NaN): 20

--- 1. Расчет корреляций ---
1. Коэффициент Фехнера: -0.6000
2. Коэффициент Пирсона: r = -0.6818, p-value = 0.0009
   95% доверительный интервал для Пирсона: [-0.8637, -0.3427]
3. Коэффициент Спирмена: r = -0.6823, p-value = 0.0009
4. Коэффициент Кенделла: tau = -0.5040, p-value = 0.0020

--- 2. Визуализация ---
Гистограммы сохранены в 'histograms.png'.
График рассеяния сохранен в 'scatter_plot.png'.

--- 3. Уравнение регрессии ---
1. Линейная: y = -0.1831 * x + 27.3961
   SSE (Сумма квадратов ошибок): 64.72
2. Квадратичная: y = -0.0033 * x^2 + 0.5396 * x + -11.8827
   SSE: 62.59
3. Гиперболическая: y = 2091.8740 / x + -11.8859
   SSE: 67.61
4. Показательная: y = 130.8014 * (0.9735 ^ x)
   SSE: 70.45
График регрессий сохранен в 'regression_plot.png'.

--- 4. Проверка уравнения регрессии ---
Лучшая модель (ми